In [1]:
import numpy as np
import  pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import Select
import matplotlib.pyplot as plt

PATH="chromedriver.exe"
driver =webdriver.Chrome(PATH)
time.sleep(2.4)

website='http://www.nepalstock.com/todaysprice'
driver.get(website)
time.sleep(2.4)
pd.set_option('display.max_rows', None)

In [2]:
def extract_data_csv(rows,columns,date_time_str):        
    data=[]
    for k in range(3,rows-3):
        rows=[]
        rows.append(date_time_str)
        for l in range(1,columns+1):
            value=driver.find_element_by_xpath('//*[@id="home-contents"]/table/tbody/tr['+str(k)+']/td['+str(l)+']').text
            rows.append(value)
        data.append(rows)
            
    df=pd.DataFrame(data=data,columns=[
                    'Date','S.N.',
                     'Traded Companies',
                 'No. Of Transaction',
                     'Max Price',
                     'Min Price',
                     'Closing Price',
                 'Traded Shares',
                 'Amount',
                 'Previous Closing',
                     'Difference Rs.'])
            
    #pulling listed stock name and sector name
    df_sym_sec=pd.read_csv('total_listed_stock.csv')
    #merging
    df=df.merge(df_sym_sec,left_on="Traded Companies",right_on="Stock Name",how="left")
    #dropping unnexessary columns
    df=df.drop(['Stock Name','S.N.'],axis=1)
            
    #making new columns
    col=['Date', 'Traded Companies', 'Stock Symbol', 'No. Of Transaction', 'Max Price',
           'Min Price', 'Closing Price', 'Traded Shares', 'Amount',
           'Previous Closing', 'Difference Rs.', 'Sector']
    df=df[col]
    df['Date']=pd.to_datetime(df.Date)
    df.to_csv('datasets/price/'+str(date_time_str)+'.csv',index=False)

In [3]:
from datetime import datetime
import calendar
from calendar import monthrange
import math


def cal_day_month():
    
    date_time_str=driver.find_element_by_xpath('//*[@id="startDateToday"]').get_attribute("value")
    date=datetime.strptime(date_time_str, '%Y-%m-%d')

    week_day=date.weekday()

    month_range=monthrange(date.year, date.month)
    print(month_range)
    rows=((month_range[1]+month_range[0])+2)/7

    start=month_range[0]+2
    end=month_range[1]
    return rows,start,end

def get_month_start(i,j,date):
    print("End of month reached :" +str(date))
    date=datetime.strptime(date, '%Y-%m-%d')
    print(i , j)
    week_day=date.weekday()
    
    if j<7:
        j=j+1
        driver.find_element_by_xpath('//*[@id="startDateToday"]').click()
        print("another month start from if")
        print(i , j)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/table/tbody/tr['+str(i)+']/td['+str(j)+']').click()
        #date_time_str=driver.find_element_by_xpath('//*[@id="startDateToday"]').get_attribute("value")
        
    else:
        i=i+1
        j=1
        driver.find_element_by_xpath('//*[@id="startDateToday"]').click()
        #time.sleep(5)
        print("another month start from else")
        print(i , j)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/table/tbody/tr['+str(i)+']/td['+str(j)+']').click()
        #time.sleep(5)
        #date_time_str=driver.find_element_by_xpath('//*[@id="startDateToday"]').get_attribute("value")
        #print(date_time_str)
        print("------")
        

def change_date(rows,start,end):
    break_me=False
    print("--------------")
    flags=1
    for i in range(1,math.ceil(rows)+1):
        
        for j in range(1,8):
            if start <= j:
                if j==6 or j==7:
                    if flags == end:
                        get_month_start(i,j,date_time_str)
                        break_me=True
                        break
                    flags =flags+1
                    continue
                driver.find_element_by_xpath('//*[@id="startDateToday"]').click()
                print(i , j)
                driver.find_element_by_xpath('/html/body/div[6]/div[1]/table/tbody/tr['+str(i)+']/td['+str(j)+']').click()
                date_time_str=driver.find_element_by_xpath('//*[@id="startDateToday"]').get_attribute("value")
            
            
                ##click filter buttom
                driver.find_element_by_xpath('//*[@id="news_info-filter"]/input[1]').click()
                time.sleep(1)
            
            
                rows=len(driver.find_elements_by_xpath('//*[@id="home-contents"]/table/tbody/tr'))
                columns=len(driver.find_elements_by_xpath('//*[@id="home-contents"]/table/tbody/tr[3]/td'))
            
                if rows < 20:
                    print("no Data for this"+str(date_time_str)+ ",public holiday")
                    print("--------")
                    if flags == end:
                        get_month_start(i,j,date_time_str)
                        break
                    flags=flags+1
                    continue
                print(date_time_str)
                print(flags)
            
                ##function to extraxt the data from table and save as csv file
                extract_data_csv(rows,columns,date_time_str)
            
            
                #code to break at the end of the month
                print("-----")
                if flags == end:
                    get_month_start(i,j,date_time_str)
                    break_me=True
                    break
                flags =flags+1
        start=0
        if break_me:
            break


In [4]:
for i in range(29):
    rows,start,end=cal_day_month()
    change_date(rows,start,end)
    print("END OF 1 MONTH")

(1, 31)
--------------
1 3
2019-01-01
1
-----
1 4
2019-01-02
2
-----
1 5
2019-01-03
3
-----
2 1
2019-01-06
6
-----
2 2
2019-01-07
7
-----
2 3
2019-01-08
8
-----
2 4
2019-01-09
9
-----
2 5
2019-01-10
10
-----
3 1
2019-01-13
13
-----
3 2
2019-01-14
14
-----
3 3
2019-01-15
15
-----
3 4
2019-01-16
16
-----
3 5
2019-01-17
17
-----
4 1
2019-01-20
20
-----
4 2
2019-01-21
21
-----
4 3
2019-01-22
22
-----
4 4
2019-01-23
23
-----
4 5
2019-01-24
24
-----
5 1
2019-01-27
27
-----
5 2
2019-01-28
28
-----
5 3
2019-01-29
29
-----
5 4
2019-01-30
30
-----
5 5
2019-01-31
31
-----
End of month reached :2019-01-31
5 5
another month start from if
5 6
END OF 1 MONTH
(4, 28)
--------------
2 1
2019-02-03
3
-----
2 2
2019-02-04
4
-----
2 3
2019-02-05
5
-----
2 4
2019-02-06
6
-----
2 5
2019-02-07
7
-----
3 1
2019-02-10
10
-----
3 2
2019-02-11
11
-----
3 3
2019-02-12
12
-----
3 4
2019-02-13
13
-----
3 5
2019-02-14
14
-----
4 1
2019-02-17
17
-----
4 2
2019-02-18
18
-----
4 3
2019-02-19
19
-----
4 4
2019-02-20
20


-----
3 2
2020-01-13
13
-----
3 3
2020-01-14
14
-----
3 4
2020-01-15
15
-----
3 5
2020-01-16
16
-----
4 1
2020-01-19
19
-----
4 2
2020-01-20
20
-----
4 3
2020-01-21
21
-----
4 4
2020-01-22
22
-----
4 5
2020-01-23
23
-----
5 1
2020-01-26
26
-----
5 2
2020-01-27
27
-----
5 3
2020-01-28
28
-----
5 4
2020-01-29
29
-----
5 5
2020-01-30
30
-----
End of month reached :2020-01-30
5 6
another month start from if
5 7
END OF 1 MONTH
(5, 29)
--------------
2 1
2020-02-02
2
-----
2 2
2020-02-03
3
-----
2 3
2020-02-04
4
-----
2 4
2020-02-05
5
-----
2 5
2020-02-06
6
-----
3 1
2020-02-09
9
-----
3 2
2020-02-10
10
-----
3 3
2020-02-11
11
-----
3 4
2020-02-12
12
-----
3 5
2020-02-13
13
-----
4 1
2020-02-16
16
-----
4 2
2020-02-17
17
-----
4 3
2020-02-18
18
-----
4 4
2020-02-19
19
-----
4 5
2020-02-20
20
-----
5 1
2020-02-23
23
-----
5 2
2020-02-24
24
-----
5 3
2020-02-25
25
-----
5 4
2020-02-26
26
-----
5 5
2020-02-27
27
-----
End of month reached :2020-02-27
5 7
another month start from else
6 1
------

2020-10-28
28
-----
5 5
2020-10-29
29
-----
End of month reached :2020-10-29
5 7
another month start from else
6 1
------
END OF 1 MONTH
(6, 30)
--------------
2 1
2020-11-01
1
-----
2 2
2020-11-02
2
-----
2 3
2020-11-03
3
-----
2 4
2020-11-04
4
-----
2 5
2020-11-05
5
-----
3 1
2020-11-08
8
-----
3 2
2020-11-09
9
-----
3 3
2020-11-10
10
-----
3 4
2020-11-11
11
-----
3 5
2020-11-12
12
-----
4 1
no Data for this2020-11-15,public holiday
--------
4 2
no Data for this2020-11-16,public holiday
--------
4 3
no Data for this2020-11-17,public holiday
--------
4 4
2020-11-18
18
-----
4 5
2020-11-19
19
-----
5 1
2020-11-22
22
-----
5 2
2020-11-23
23
-----
5 3
2020-11-24
24
-----
5 4
2020-11-25
25
-----
5 5
2020-11-26
26
-----
6 1
2020-11-29
29
-----
6 2
2020-11-30
30
-----
End of month reached :2020-11-30
6 2
another month start from if
6 3
END OF 1 MONTH
(1, 31)
--------------
1 3
2020-12-01
1
-----
1 4
2020-12-02
2
-----
1 5
2020-12-03
3
-----
2 1
2020-12-06
6
-----
2 2
2020-12-07
7
-----
2 3


## Program to fetch single day data

In [267]:
date=driver.find_element_by_xpath('//*[@id="startDateToday"]').get_attribute("value")
rows=len(driver.find_elements_by_xpath('//*[@id="home-contents"]/table/tbody/tr'))
columns=len(driver.find_elements_by_xpath('//*[@id="home-contents"]/table/tbody/tr[3]/td'))

print(date)

data=[]
for i in range(3,rows-3):
    rows=[]
    rows.append(date)
    for j in range(1,columns+1):
        value=driver.find_element_by_xpath('//*[@id="home-contents"]/table/tbody/tr['+str(i)+']/td['+str(j)+']').text
        rows.append(value)
    
    data.append(rows)

    
df=pd.DataFrame(data=data,columns=[
'Date','S.N.',
 'Traded Companies',
 'No. Of Transaction',
 'Max Price',
 'Min Price',
 'Closing Price',
 'Traded Shares',
 'Amount',
 'Previous Closing',
 'Difference Rs.'])

df_sym_sec=pd.read_csv('total_listed_stock.csv')

df=df.merge(df_sym_sec,left_on="Traded Companies",right_on="Stock Name",how="left")

df=df.drop(['Stock Name','S.N.'],axis=1)

col=['Date', 'Traded Companies', 'Stock Symbol', 'No. Of Transaction', 'Max Price',
       'Min Price', 'Closing Price', 'Traded Shares', 'Amount',
       'Previous Closing', 'Difference Rs.', 'Sector']

df=df[col]

df['Date']=pd.to_datetime(df.Date)

df.to_csv('datasets/price/'+str(date)+'.csv',index=False)



2016-07-31
